# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 17 2023 11:04AM,261183,10,PRF-58162,Bio-PRF,Released
1,Apr 17 2023 11:04AM,261183,10,PRF-58165,Bio-PRF,Released
2,Apr 17 2023 11:04AM,261183,10,PRF-58168,Bio-PRF,Released
3,Apr 17 2023 11:04AM,261183,10,PRF-58171,Bio-PRF,Released
4,Apr 17 2023 11:04AM,261183,10,PRF-58218,Bio-PRF,Released
5,Apr 17 2023 11:04AM,261183,10,PRF-58221,Bio-PRF,Released
6,Apr 17 2023 11:04AM,261183,10,PRF-58229,Bio-PRF,Released
7,Apr 17 2023 11:04AM,261183,10,PRF-58237,Bio-PRF,Released
8,Apr 17 2023 11:04AM,261183,10,PRF-58242,Bio-PRF,Released
9,Apr 17 2023 11:04AM,261183,10,PRF-58247,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,261184,Released,10
25,261185,Released,1
26,261186,Released,1
27,261187,Released,2
28,261188,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261184,NaN,NaN,10.0
261185,NaN,NaN,1.0
261186,NaN,NaN,1.0
261187,NaN,NaN,2.0
261188,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261145,1.0,11.0,2.0
261146,0.0,0.0,1.0
261147,0.0,0.0,1.0
261152,0.0,1.0,0.0
261156,0.0,10.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261145,1,11,2
261146,0,0,1
261147,0,0,1
261152,0,1,0
261156,0,10,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261145,1,11,2
1,261146,0,0,1
2,261147,0,0,1
3,261152,0,1,0
4,261156,0,10,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261145,1,11,2
1,261146,,,1
2,261147,,,1
3,261152,,1,
4,261156,,10,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 17 2023 11:04AM,261183,10,Bio-PRF
46,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.
47,Apr 17 2023 10:58AM,261187,19,"Methapharm, Inc."
49,Apr 17 2023 10:57AM,261186,10,MedStone Pharma LLC
50,Apr 17 2023 10:54AM,261185,10,Emerginnova
51,Apr 17 2023 10:45AM,261184,10,MedStone Pharma LLC
61,Apr 17 2023 10:42AM,261182,10,Bio-PRF
62,Apr 17 2023 10:12AM,261179,19,"GCH Granules USA, Inc."
63,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc."
66,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 17 2023 11:04AM,261183,10,Bio-PRF,,,46
1,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,,,1
2,Apr 17 2023 10:58AM,261187,19,"Methapharm, Inc.",,,2
3,Apr 17 2023 10:57AM,261186,10,MedStone Pharma LLC,,,1
4,Apr 17 2023 10:54AM,261185,10,Emerginnova,,,1
5,Apr 17 2023 10:45AM,261184,10,MedStone Pharma LLC,,,10
6,Apr 17 2023 10:42AM,261182,10,Bio-PRF,,,1
7,Apr 17 2023 10:12AM,261179,19,"GCH Granules USA, Inc.",,,1
8,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",,,3
9,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:04AM,261183,10,Bio-PRF,46,,
1,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,1,,
2,Apr 17 2023 10:58AM,261187,19,"Methapharm, Inc.",2,,
3,Apr 17 2023 10:57AM,261186,10,MedStone Pharma LLC,1,,
4,Apr 17 2023 10:54AM,261185,10,Emerginnova,1,,
5,Apr 17 2023 10:45AM,261184,10,MedStone Pharma LLC,10,,
6,Apr 17 2023 10:42AM,261182,10,Bio-PRF,1,,
7,Apr 17 2023 10:12AM,261179,19,"GCH Granules USA, Inc.",1,,
8,Apr 17 2023 9:56AM,261177,10,"Citieffe, Inc.",3,,
9,Apr 17 2023 9:48AM,261175,10,Eye Pharma Inc,5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:04AM,261183,10,Bio-PRF,46,,
1,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,1,,
2,Apr 17 2023 10:58AM,261187,19,"Methapharm, Inc.",2,,
3,Apr 17 2023 10:57AM,261186,10,MedStone Pharma LLC,1,,
4,Apr 17 2023 10:54AM,261185,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:04AM,261183,10,Bio-PRF,46.0,NaN,NaN
1,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,1.0,NaN,NaN
2,Apr 17 2023 10:58AM,261187,19,"Methapharm, Inc.",2.0,NaN,NaN
3,Apr 17 2023 10:57AM,261186,10,MedStone Pharma LLC,1.0,NaN,NaN
4,Apr 17 2023 10:54AM,261185,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 17 2023 11:04AM,261183,10,Bio-PRF,46.0,0.0,0.0
1,Apr 17 2023 10:59AM,261188,10,Eywa Pharma Inc.,1.0,0.0,0.0
2,Apr 17 2023 10:58AM,261187,19,"Methapharm, Inc.",2.0,0.0,0.0
3,Apr 17 2023 10:57AM,261186,10,MedStone Pharma LLC,1.0,0.0,0.0
4,Apr 17 2023 10:54AM,261185,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4178768,167.0,10.0,0.0
15,261169,8.0,13.0,0.0
18,261161,1.0,0.0,0.0
19,1566976,8.0,11.0,1.0
22,261152,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4178768,167.0,10.0,0.0
1,15,261169,8.0,13.0,0.0
2,18,261161,1.0,0.0,0.0
3,19,1566976,8.0,11.0,1.0
4,22,261152,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,167.0,10.0,0.0
1,15,8.0,13.0,0.0
2,18,1.0,0.0,0.0
3,19,8.0,11.0,1.0
4,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,167.0
1,15,Released,8.0
2,18,Released,1.0
3,19,Released,8.0
4,22,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,22
Status,,,,,
Completed,0.0,0.0,0.0,1.0,0.0
Executing,10.0,13.0,0.0,11.0,1.0
Released,167.0,8.0,1.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,22
0,Completed,0.0,0.0,0.0,1.0,0.0
1,Executing,10.0,13.0,0.0,11.0,1.0
2,Released,167.0,8.0,1.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,22
0,Completed,0.0,0.0,0.0,1.0,0.0
1,Executing,10.0,13.0,0.0,11.0,1.0
2,Released,167.0,8.0,1.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()